## StreamStats API Scraper Automatic

__Description__: Tool to automatically run the [USGS StreamStats tool](https://www.usgs.gov/mission-areas/water-resources/science/streamstats-streamflow-statistics-and-spatial-analysis-tools?qt-science_center_objects=0#qt-science_center_objects) for multiple points within a catchment and return the flow frequency curves and subcatchment boundaries. For additional details, see the [StreamStats Automation Wiki](https://github.com/Dewberry/usgs-tools/wiki/StreamStats-Automation).

__Input__: A shapefile containing the latitude and longitude of points on the stream grid for the specified state (confluence and main stem locations).

__Output__: GeoJSON file containing the delinated catchment boundary and flow frequency data for each point, as well as a CSV file containing the flow frequency data.

*Authors*: sputnam@Dewberry.com & slawler@Dewberry.com

### Load libraries and Python options:

In [1]:
import os
import sys
sys.path.append('scripts')
from stream_stats import*
import geopandas as gpd
from geojson import dump

### Specify the state abbreviation and location of the shapefile: 

In [2]:
state='MD' #The state abbreviation

path=r'C:\Users\sputnam\Documents\FEMAFloodPlain\ArcGIS\Data\Converted_Graphics_3.shp' #The location of the shapefile containing the lat/lon of the catchment outlets

##### Load the shapefile:

In [3]:
gdf=gpd.read_file(path) #Read the shapefile as a geopandas dataframe

geom=gdf.geometry #Extract the shapley geometry for the outlets in the shapefile

print(geom.head(2))

0    POINT (-77.04582686669082 38.95039400462227)
1    POINT (-77.05163009280658 38.98614966541195)
Name: geometry, dtype: object


### Run the API tool for each point:

In [4]:
pp_dic = {} #Dictionary to store the outlet flow frequency data dictionaries

watershed_poly_dic= {} #Dictionary to store the catchment polygons (catchment boundaries)

pp_fail=[] #List to store outlet locations whose flow frequency/catchment polygons were not calculated

pp_dic, watershed_poly_dic=snappoint_analysis(geom, state, status=False) #Run the snappoint function for all catchment outlets within the shapefile and for the specified state. Option: set status=False to hide print statements

-77.04582686669082 38.95039400462227
Fetched Peak Flows
-77.05163009280658 38.98614966541195
Fetched Peak Flows
-77.08791002352272 39.02925570374873
Fetched Peak Flows
-77.08850690312643 39.039659679463895
Fetched Peak Flows
-77.05096664485558 38.94574262580144
Fetched Peak Flows
-77.0428924240803 38.98763083850119
Fetched Peak Flows
-77.05672268248004 38.90366211707585
Fetched Peak Flows
-77.05908382668044 38.91585781487872
Fetched Peak Flows
-77.0558574040915 38.91910501171472
Fetched Peak Flows
-77.04729454907128 38.93541954804293
Fetched Peak Flows
-77.05099726345227 38.93308121236602
Fetched Peak Flows
-77.05225650965485 38.93819437776069
Fetched Peak Flows
-77.04968187444345 38.9443603434405
Fetched Peak Flows
-77.05115242417283 38.9454841569903
Fetched Peak Flows
-77.04166512711268 38.960312030555976
Fetched Peak Flows
-77.043697885877 38.97146471911136
Fetched Peak Flows
-77.04147255092862 38.977653361999565
Fetched Peak Flows
-77.04265655549406 38.98759063282516
Fetched Peak F

### Construct a summary table of the flow frequency data for each outlet:

In [5]:
ffdata=ff_summary(pp_dic) #Run this function to construct the summary table for all outlet locations

ffdata.head()    

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,41,42,43
RI,,,,,,,,,,,,,,,,,,,,,
1.25,2092.204261,2005.783493,1556.172340,394.454387,133.670903,230.688773,1455.697927,115.970397,138.521504,157.567432,...,59.186866,141.358736,20.010962,8.744872,88.562532,154.746088,105.338501,135.764335,159.094427,1371.094513
1.50,2670.650421,2553.785078,2004.086140,474.319874,179.742639,264.388109,1829.162182,134.709483,160.314564,202.823967,...,69.713208,172.218148,28.251335,12.878904,107.570067,178.682118,129.068575,157.188935,195.024313,1776.993068
2.00,3288.227422,3143.145067,2484.085735,583.712172,233.032846,322.059609,2249.079831,166.250453,197.170575,244.341560,...,87.164774,215.582144,38.125919,17.816986,135.281866,219.289580,162.469907,193.401711,244.304114,2210.890486
5.00,5567.755352,5306.637811,4284.689433,935.741949,448.485941,486.518174,3490.346627,259.476690,305.062555,354.231123,...,140.610715,359.194833,81.658363,41.219674,226.429297,337.445139,274.488728,299.526871,409.778786,3858.097608
10.00,7537.771063,7173.304481,5864.947471,1233.461760,656.319004,621.163401,4602.066707,337.914196,395.188069,464.376055,...,186.813014,484.936070,127.409138,67.271382,306.802256,435.701140,373.716602,388.248847,555.143434,5316.990890


###  Save the results:

##### As a CSV:

In [6]:
cpath=os.path.join(os.getcwd(),'results') #Location to save the results

if os.path.isdir(cpath)==False: #If the desired path does not exist, create it.
    os.mkdir(cpath)    
    
ffdata.to_csv(os.path.join(cpath,'flow_frequency_data.csv')) #Save the results as a csv

##### As a geojson:

In [7]:
for i in pp_dic.keys():
    watershed_poly_dic[i]['features'][0]['ffcurve']=pp_dic[i]
    
with open('results\catchment_poly.geojson', 'w') as f:
   dump(watershed_poly_dic, f)        

# END